In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os
import tensorflow as tf
print(f"gpu: { len(tf.config.list_physical_devices('GPU')) }")
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
from tensorflow.keras.optimizers import Adam
import math
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from vae_adapted import VAEAdapted

gpu: 1


In [2]:
def load_data_for_model(model_name, batch_size=64):
    labels = np.load('./data/LFSD_labels.npy')
    depths = np.load('./data/LFSD_depths_repeated_%s_feat.npy' % model_name)
    imgs = np.load('./data/LFSD_imgs_%s_feat.npy' % model_name)
    masks = np.load('./data/LFSD_masks_single.npy')
    idx = np.random.permutation(len(labels))
    batch_idxs = [idx[i:i + batch_size] for i in range(0, len(labels), batch_size)]
    dataset = []
    for idx in batch_idxs:
        img_batch, depth_batch, mask_batch = imgs[idx], depths[idx], masks[idx]
        dataset.append((img_batch, depth_batch, mask_batch))
    train_dataset, test_dataset = train_test_split(dataset, test_size=0.3)
    print("Train dataset contains %d batches of %d samples each" % (len(train_dataset), batch_size))
    print("Test dataset contains %d batches of %d samples each" % (len(test_dataset), batch_size))
    return train_dataset, test_dataset

In [3]:
def merge_images(image_batch, size):
    h,w = image_batch.shape[1], image_batch.shape[2]
    img = np.zeros((int(h*size[0]), w*size[1]))
    for idx, im in enumerate(image_batch):
        im = np.squeeze(im, axis=2)
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = im
    return img

In [4]:
def train_round(train_dataset, test_dataset, learning_rate, model_name, epochs):
    latent_lookup = {
        'inception': 2048,
        'vgg': 512,
        'efficientnet': 1280,
        'mobilenet': 1280,
        'resnet': 2048,
    }
    latent_dim = latent_lookup[model_name]
    vae = VAEAdapted(latent_dim)
    vae.compile(optimizer=Adam(learning_rate))
    epochs = epochs
    # Training Step
    losses_across_epochs = {
        "loss": [],
        "reconstruction_loss": [],
        "kl_loss": [],
    }
    batch_num = len(train_dataset)
    for i in range(epochs):

        for k, v in losses_across_epochs.items():
            losses_across_epochs[k].append(0)
        for data in train_dataset:
            cur_loss = vae.train_step(data)
            for k, v in cur_loss.items():
                losses_across_epochs[k][-1] += cur_loss[k].numpy() / batch_num
            generated_image = vae.sample(data)
        print(f"Epoch {i} Total loss: { losses_across_epochs['loss'][-1]}")
        im_merged = merge_images(generated_image.numpy(), [8,8])
        plt.imsave('./images/vae_adapted/%d.png' % i, im_merged, cmap='gray')
    for k, v in losses_across_epochs.items():
        np.save('./results/vae_adapted/%s_%s' % (model_name, k), np.array(v))
    # Testing Step
    test_loss = 0
    for i, data in enumerate(test_dataset):
        _, _, mask_batch = data
        generated_image = vae.sample(data)
        reconstruction_loss = tf.reduce_sum(
            tf.keras.losses.binary_crossentropy(mask_batch, generated_image), [1,2]
        )
        test_loss += tf.reduce_mean(reconstruction_loss).numpy()
        im_merged = merge_images(generated_image.numpy(), [8,8])
        plt.imsave('./images/vae_adapted/test_batch_%d.png' % i, im_merged, cmap='gray')
        
    test_loss = test_loss / len(test_dataset)
    np.save('./results/vae_adapted/%s_test_loss' % model_name, np.array([test_loss]))
    return vae

In [5]:
def get_encoding_for_model(vae, model_name):
    from skimage.transform import resize
    from tensorflow.keras import datasets

    train_dataset, test_dataset = None, None
 
    train_feats = np.load('./data/CIFAR100_%s_train_feat.npy' % model_name)
    test_feats = np.load('./data/CIFAR100_%s_test_feat.npy' % model_name)

    train_result, _, _ = vae.encode(train_feats[:128], tf.random.normal(train_feats[:128].shape))
    for i in range(128, len(train_feats), 128):
        activation, _, _ = vae.encode(train_feats[i:i+128], tf.random.normal(train_feats[i:i+128].shape))
        train_result = tf.concat((train_result, activation), axis=0)
    np.save('./data/CIFAR100_vae_adapted_%s_encoding_train.npy' % model_name, np.array(train_result))

    test_result, _, _ = vae.encode(test_feats[:128], tf.random.normal(test_feats[:128].shape))
    for i in range(128, len(test_feats), 128):
        activation, _, _ = vae.encode(test_feats[i:i+128], tf.random.normal(test_feats[i:i+128].shape))
        test_result = tf.concat((test_result, activation), axis=0)
    np.save('./data/CIFAR100_vae_adapted_%s_encoding_test.npy' % model_name, np.array(test_result))


In [6]:
%%time

learning_rate = 1e-4
epochs = 100
for model_name in ['efficientnet','inception', 'resnet', 'mobilenet', 'vgg']:
    train_dataset, test_dataset = load_data_for_model(model_name)
    trained_model = train_round(train_dataset, test_dataset, learning_rate, model_name, epochs)
    print("Gen encoding...")
    get_encoding_for_model(trained_model, model_name)

Train dataset contains 9 batches of 64 samples each
Test dataset contains 5 batches of 64 samples each
Epoch 0 Total loss: 6701287708.444446
Epoch 1 Total loss: 1822572771.5555556
Epoch 2 Total loss: 1087946837.3333335
Epoch 3 Total loss: 777895637.3333335
Epoch 4 Total loss: 606177358.2222222
Epoch 5 Total loss: 496903875.5555555
Epoch 6 Total loss: 421198531.5555556
Epoch 7 Total loss: 365632465.7777778
Epoch 8 Total loss: 323101504.0
Epoch 9 Total loss: 289487164.4444444
Epoch 10 Total loss: 262235896.8888889
Epoch 11 Total loss: 239691393.77777782
Epoch 12 Total loss: 220732826.6666667
Epoch 13 Total loss: 204569015.1111111
Epoch 14 Total loss: 190625360.0
Epoch 15 Total loss: 178474538.6666667
Epoch 16 Total loss: 167791751.1111111
Epoch 17 Total loss: 158326161.7777778
Epoch 18 Total loss: 149881168.0
Epoch 19 Total loss: 142300074.66666666
Epoch 20 Total loss: 135456757.33333334
Epoch 21 Total loss: 129248542.22222222
Epoch 22 Total loss: 123590904.88888888
Epoch 23 Total loss: 

Epoch 3 Total loss: 3223564.916666667
Epoch 4 Total loss: 3066423.4166666665
Epoch 5 Total loss: 2877127.361111111
Epoch 6 Total loss: 2680900.0
Epoch 7 Total loss: 2494427.5833333335
Epoch 8 Total loss: 2323502.0555555555
Epoch 9 Total loss: 2168953.5833333335
Epoch 10 Total loss: 2033768.3055555557
Epoch 11 Total loss: 1917383.1944444445
Epoch 12 Total loss: 1816448.5555555555
Epoch 13 Total loss: 1728045.9305555557
Epoch 14 Total loss: 1649026.1805555553
Epoch 15 Total loss: 1577662.2777777778
Epoch 16 Total loss: 1512059.1805555555
Epoch 17 Total loss: 1452265.9722222222
Epoch 18 Total loss: 1397472.7916666665
Epoch 19 Total loss: 1347080.236111111
Epoch 20 Total loss: 1300477.4861111115
Epoch 21 Total loss: 1257620.4166666665
Epoch 22 Total loss: 1219028.1666666667
Epoch 23 Total loss: 1183870.7083333333
Epoch 24 Total loss: 1150981.5277777778
Epoch 25 Total loss: 1119828.4305555555
Epoch 26 Total loss: 1090188.3611111112
Epoch 27 Total loss: 1062228.625
Epoch 28 Total loss: 10360

Epoch 9 Total loss: 11663422.333333332
Epoch 10 Total loss: 10734816.555555556
Epoch 11 Total loss: 9962376.444444444
Epoch 12 Total loss: 9310329.111111112
Epoch 13 Total loss: 8752985.0
Epoch 14 Total loss: 8271071.944444444
Epoch 15 Total loss: 7850385.555555554
Epoch 16 Total loss: 7479714.166666668
Epoch 17 Total loss: 7150564.277777778
Epoch 18 Total loss: 6856487.166666666
Epoch 19 Total loss: 6592157.888888889
Epoch 20 Total loss: 6353387.888888889
Epoch 21 Total loss: 6136516.333333333
Epoch 22 Total loss: 5938670.166666666
Epoch 23 Total loss: 5757357.999999999
Epoch 24 Total loss: 5590640.277777778
Epoch 25 Total loss: 5436813.555555555
Epoch 26 Total loss: 5294514.111111111
Epoch 27 Total loss: 5162458.111111111
Epoch 28 Total loss: 5039571.166666666
Epoch 29 Total loss: 4924826.833333334
Epoch 30 Total loss: 4817510.833333334
Epoch 31 Total loss: 4716937.611111111
Epoch 32 Total loss: 4622615.944444444
Epoch 33 Total loss: 4533849.055555556
Epoch 34 Total loss: 4450161.333